In [1]:
import pandas as pd
import numpy as np
from openeye import oechem, oedepict
import oenotebook as oenb
from IPython.display import HTML, display
from base64 import b64encode

In [2]:
def get_labeled_mol(smiles, label='heavy'):
    """
    returns an OEMol with heavy atoms labeled with a specific indice
    """
    mol = oechem.OEMol()
    if not oechem.OESmilesToMol(mol, smiles):
        print("Couldn't parse smiles (%s) returning None" % smiles)
        return None
    
    for idx, a in enumerate(mol.GetAtomIter(oechem.OEIsHeavy())):
        a.SetData('heavy', idx+1)
    
    return mol

def get_total_charge(mol):
    """
    Calculates total formal charge of a molecule.
    
    Input
    mol: oechem.OEMol() object
    """
    total_charge = 0
    for a in mol.GetAtomIter():
        total_charge += a.GetFormalCharge()
    return total_charge

def compare_total_charge(mol1, mol2):
    """
    Compares total formal charge of two SMILES strings.
    Returns True of they have equal formal charge.
    """
    total_charge1 = get_total_charge(mol1)
    total_charge2 = get_total_charge(mol2)
    if total_charge1 == total_charge2:
        return True
    else:
        return False
    
def calculate_total_charge_difference(mol1, mol2):
    charge_difference = get_total_charge(mol1) - get_total_charge(mol2)
    return charge_difference

def count_heavy_atoms(mol):
    """
    Returns number of heavy atoms in a molecule.
    """
    heavy_atom_count = 0
    for idx, atom in enumerate(mol.GetAtomIter()):
        heavy_atom_count = idx + 1
        #print(atom, atom.GetData()) # Target mol has labelled heavy atoms
    return heavy_atom_count

def get_total_hydrogen_count(mol):
    """
    Calculates total number of hydrogens in a structure.
    """
    hydrogen_count = 0
    
    for idx, atom in enumerate(mol.GetAtomIter()):
        hydrogen_count += atom.GetTotalHCount()
    
    return hydrogen_count

def get_mcss(pattern, target):
    """
    Finds maximum common substructure based on atomic number criteria 
    
    Returns the match as a mol object and a dictionary that maps pattern heavy atom labels to target labels .
    
    MCSS should result in only 1 match.
    """
    atomexpr = oechem.OEExprOpts_AtomicNumber
    bondexpr = oechem.OEExprOpts_EqSingleDouble # single or double bonds are considered identical

    #bondexpr = oechem.OEExprOpts_DefaultBonds
    #atomexpr = oechem.OEExprOpts_DefaultAtoms 
    
    # create maximum common substructure object
    mcss = oechem.OEMCSSearch(pattern, atomexpr, bondexpr, oechem.OEMCSType_Exhaustive)
    
    # set scoring function
    #mcss.SetMCSFunc(oechem.OEMCSMaxAtoms())
    mcss.SetMCSFunc(oechem.OEMCSMaxAtomsCompleteCycles())

    # ignore matches smaller than 6 atoms
    mcss.SetMinAtoms(6)
    unique = True
    
    # create a dictionary that maps pattern heavy atom labels to target labels
    label_dict = {}

    # loop over matches
    for i, match in enumerate(mcss.Match(target, unique)):
        count = i + 1
        print ("Match %d:" % (count))
        
        print ("pattern atoms:", end=" ")
        for ma in match.GetAtoms():
            print (ma.pattern.GetIdx(), end=" ")
        print ("\ntarget atoms: ", end=" ")
        for ma in match.GetAtoms():
            print (ma.target.GetIdx(), end=" ")
        print()
        
        print ("\npattern heavy atom labels:", end=" ")
        for ma in match.GetAtoms():
            print (ma.pattern.GetData("heavy"), end=" ")
        print ("\ntarget heavy atom labels: ", end=" ")
        for ma in match.GetAtoms():
            print (ma.target.GetData("heavy"), end=" ")
        print()
        
        # record matching pattern and target labels in a dictionary
        for ma in match.GetAtoms():
            label_dict[ma.pattern.GetData("heavy")] = ma.target.GetData("heavy")

        # create match subgraph
        m = oechem.OEGraphMol()
        oechem.OESubsetMol(m, match, True)

        print ("\nmatch smiles =", oechem.OEMolToSmiles(m))

    # check if there is only single match
    if (count != 1):
        print("Warning! There is multiple matches.")
    else:
        print("Exactly one match.")
        
    return m, label_dict, mcss

def get_labelled_heavy_atom_Hcount(mol, heavy_atom_label, data_section = "heavy"):
    """
    finds the heavy atom with the same label in a molecule and returns its H-count
    """
    query_label = heavy_atom_label
    for i, atom in enumerate(mol.GetAtomIter()):
        pattern_label = atom.GetData(data_section)
        if pattern_label == query_label:
            Hcount = atom.GetTotalHCount()
            return Hcount
    # if not found
    return np.NaN

def check_how_many_heavy_atoms_changed_protonation_state(pattern, target, label_dict):
    """
    Returns the number of labeled heavy atoms that have different number of bound hydrogens between
    two proposed microstates of the same molecule.
    """
    
    # create numpy arrays to store heavy atom hydrogen counts
    pattern_HCount_array = np.zeros(count_heavy_atoms(pattern))
    target_HCount_array = np.zeros(count_heavy_atoms(target))
    #print("Pattern number of heavy atoms: ", count_heavy_atoms(pattern))
    #print("Target number of heavy atoms:  ", count_heavy_atoms(target))
    #print("label_dict:")
    #print(label_dict)
    #print("len pattern_HCount_array: ", len(pattern_HCount_array))
    #print("len target_HCount_array: ", len(target_HCount_array))

    # iterate over label dictionary to and populate Hcount arrays
    for ii, (key, value) in enumerate(label_dict.items()):
        #print(ii, key, value)
        p_label = key
        t_label = value

        # find the heavy atom with the same label in pattern molecule and get H-count
        Hcount = get_labelled_heavy_atom_Hcount(pattern, p_label)
        #print("ii: ", ii)
        pattern_HCount_array[ii] = Hcount

        # find the heavy atom with the same label in target molecule and get H-count
        Hcount = get_labelled_heavy_atom_Hcount(target, t_label)
        target_HCount_array[ii] = Hcount

    print("Pattern Hcount: ", pattern_HCount_array)
    print("Target Hcount: ", target_HCount_array)

    # calculate difference of 2 arrays
    Hcount_difference_array = np.subtract(pattern_HCount_array,target_HCount_array)
    print("Difference in Hcount:", Hcount_difference_array )

    # calculate the number of heavy atoms protonated or deprotonated
    prot_deprot_heavy_atom_count = 0
    for diff in Hcount_difference_array:
        if diff != 0.0:
            prot_deprot_heavy_atom_count += 1

    print("Number of heavy atoms with different number of hydrogens: ", prot_deprot_heavy_atom_count)
    return prot_deprot_heavy_atom_count

def is_physical_microstate_pair(pattern, target):
    
    # Compare number of heavy atoms
    difference_of_heavy_atom_number = count_heavy_atoms(pattern) - count_heavy_atoms(target)
    if difference_of_heavy_atom_number != 0:
        print("Not microstates of the same molecule.")
        return False
    
    # Compare total charge
    total_charge_difference = calculate_total_charge_difference(pattern, target)
    if abs(total_charge_difference) != 1:
        print("Not a physical microstate pair. Total charge difference is ", total_charge_difference)
        return False
    
    # Find MCSS of two molecules and find equivalent heavy atoms
    label_dictionary = {}
    m, label_dictionary, mcss = get_mcss(pattern, target)
    #print("Length of label_dictionary: ", len(label_dictionary) )
    
    # how many heavy atoms have different protonation state?
    prot_deprot_heavy_atom_count = check_how_many_heavy_atoms_changed_protonation_state(pattern, target, label_dictionary)
    if prot_deprot_heavy_atom_count == 1 :
        print("These two microstates create a physical microscopic pKa pair.")
        return True
    else:
        print("Not a physical microstate pair.")
        return False

In [3]:
def create_microstate_pairs_dataframe(molecule_ID):
    df_ms = pd.read_csv(path_to_microstates_csv)
    microstate_IDs = df_ms["microstate ID"] 
    
    df_ms_pairs = pd.DataFrame()
    df_ms_pairs["microstate ID 1"] = np.NaN
    df_ms_pairs["microstate ID 2"] = np.NaN
    df_ms_pairs["physical"] = np.NaN

    for i, ms1 in enumerate(microstate_IDs):
        for j, ms2 in enumerate(microstate_IDs):
            if j > i:
                df_pair = pd.DataFrame([[ms1, ms2, np.NaN]], columns=["microstate ID 1", "microstate ID 2", "physical"] )
                df_ms_pairs = df_ms_pairs.append(df_pair)

    df_ms_pairs = df_ms_pairs.reset_index(drop=True)
    
    return df_ms_pairs

def microstateID_to_smiles(molecule_ID, ms_ID):
    """
    Returns the canonical isomeric smiles of given ms_ID, reading from SMXX_microstates.csv file.
    """
    df_ms = pd.read_csv(path_to_microstates_csv)
    #print(df_ms)
    smiles = df_ms.loc[df_ms["microstate ID"] == ms_ID]["canonical isomeric SMILES"].values[0]
    return smiles

In [4]:
molecule_ID = "SM18"
path_to_microstates_csv = "./microstates/"+molecule_ID+"_microstates.csv"

df_microstate_pairs = create_microstate_pairs_dataframe(molecule_ID)
df_microstate_pairs

,microstate ID 1,microstate ID 2,physical
0,SM18_micro001,SM18_micro002,NaN
1,SM18_micro001,SM18_micro003,NaN
2,SM18_micro001,SM18_micro004,NaN
3,SM18_micro001,SM18_micro005,NaN
4,SM18_micro001,SM18_micro007,NaN
5,SM18_micro001,SM18_micro009,NaN
6,SM18_micro001,SM18_micro010,NaN
7,SM18_micro001,SM18_micro011,NaN
8,SM18_micro001,SM18_micro013,NaN
9,SM18_micro001,SM18_micro016,NaN


In [5]:
for i in range(df_microstate_pairs.shape[0]):
    # get 2 microstate IDs
    microstate_ID1 = df_microstate_pairs.loc[i, "microstate ID 1"]
    print(microstate_ID1)
    microstate_ID2 = df_microstate_pairs.loc[i, "microstate ID 2"]
    print(microstate_ID2)
    
    # get smiles strings of 2 molecules
    smiles1 = microstateID_to_smiles(molecule_ID, microstate_ID1)
    print(smiles1)
    smiles2 = microstateID_to_smiles(molecule_ID, microstate_ID2)
    print(smiles2)
    
    # get oemol objects of 2 molecules
    pattern = get_labeled_mol(smiles1)
    target = get_labeled_mol(smiles2)
    
    # record if microstate pair is physical or not
    is_physical = is_physical_microstate_pair(pattern, target)
    df_microstate_pairs.loc[i, "physical"] = is_physical
    
    print("Done!\n\n")
    
df_microstate_pairs

SM18_micro001
SM18_micro002
c1ccc2c(c1)c(=O)[nH]c(n2)CCC(=O)/[NH+]=C/3\N=CC(S3)Cc4ccc(c(c4)F)F
c1ccc2c(c1)c(=O)[n-]c([nH+]2)CCC(=O)Nc3ncc(s3)Cc4ccc(c(c4)F)F
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

match smiles = c1ccc2c(c1)c(=O)[n-]c([nH+]2)CCC(=O)Nc3ncc(s3)Cc4ccc(c(c4)F)F
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  2.  0.  0.  1.  0.  0.
  1.  1.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.  2.  2.  0.  0.  1.  0.  0.
  1.  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0. 

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

match smiles = c1ccc2c(c1)c(=O)nc([nH]2)CCC(=[OH+])[N-]c3ncc(s3)Cc4ccc(c(c4)F)F
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  2.  0.  0.  1.  0.  0.
  1.  1.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.  2.  2.  0.  1.  0.  0.  0.
  1.  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0. -1.  1.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

match smiles = c1ccc2c(c1)c(=O)[nH]c(n2)CCC(=O)/N=c\3/[nH]cc(s3)Cc4ccc(c(c4)F)F
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  2.  0.  0.  1.  0.  0.
  1.  1.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  2.  0.  0.  0.  0.  1.
  1.  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. -1.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 30 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

match smiles = c1ccc2c(c1)c(nc(n2)CCC(=O)/N=C/3\[NH+]=CC(S3)Cc4ccc(c(c4)F)F)[O-]
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  2.  0.  0.  1.  0.  0.
  1.  1.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  2.  2.  0.  0.  0.  0.  1.
  1.  1.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0. -1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens:  3
Not a physical microstate pair.
Done!


SM18_micro001
SM18_micro062
c1ccc2c(c1)c(=O)[nH]c(n2)CCC(=O)/[NH+]=C/3\N=CC(S3)Cc4ccc(c(c4)F)F
c1ccc2c(c1)c(=O)[nH]c(n2)C/C=C(/N=c/3\[nH]cc(s3)Cc4ccc(c(c4)F

target atoms:  0 1 2 3 4 5 6 29 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 30 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

match smiles = c1ccc2c(c1)c(nc(n2)CCC(=O)/N=C/3\N=CC(S3)Cc4ccc(c(c4)F)F)[O-]
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.  2.  2.  0.  0.  1.  0.  0.
  1.  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  2.  2.  0.  0.  0.  0.  0.
  1.  1.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.
  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens:  3
Not a physical microstate pair.
Done!


SM18_micro002
SM18_micro013
c1ccc2c(c1)c(=O)[n-]c([nH+]2)CCC(=O

...

target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

match smiles = c1ccc2c(c1)c(=O)[nH+]c([nH]2)C/C=C(/N=c/3\[nH]cc(s3)Cc4ccc(c(c4)F)F)\[O-]
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  1.  0.  0.  0.  1.  1.
  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  2.  1.  0.  0.  0.  1.  1.
  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens:  1
These two microstates create a physical microscopic pKa pair.
Done!


SM18_micro062
SM18_mi

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target atoms:  0 1 2 3 4 5 6 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 7 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 8 

match smiles = c1ccc2c(c1)c(=O)[nH]c(n2)C/C=C(/Nc3ncc(s3)Cc4ccc(c(c4)F)F)\[O-]
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  2.  1.  0.  0.  0.  1.  1.  0.
  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  1.  0.  0.  2.  1.  0.  1.  0.  0.  1.  0.
  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0. -1.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens:

Number of heavy atoms with different number of hydrogens:  3
Not a physical microstate pair.
Done!


SM18_micro068
SM18_micro071
c1ccc2c(c1)c(=O)[nH+]c([nH]2)C/C=C(/N=c/3\[nH]cc(s3)Cc4ccc(c(c4)F)F)\[O-]
c1ccc2c(c1)c(nc(n2)CCC(=O)Nc3ncc(s3)Cc4ccc(c(c4)F)F)[O-]
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target atoms:  0 1 2 3 4 5 6 29 7 8 9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 13 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
target heavy atom labels:  1 2 3 4 5 6 7 30 8 9 10 11 12 13 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 14 

match smiles = c1ccc2c(c1)c(nc(n2)CCC(=O)Nc3ncc(s3)Cc4ccc(c(c4)F)F)[O-]
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  2.  1.  0.  0.  0.  1.  1.
  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  2.  2.  0.  1.  0.  0.  1.

target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

match smiles = c1ccc2c(c1)c(=[OH+])[nH]c(n2)CCC(=O)Nc3ncc(s3)Cc4ccc(c(c4)F)F
Exactly one match.
Pattern Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  2.  2.  0.  1.  0.  0.  0.
  1.  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Target Hcount:  [ 1.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.  2.  2.  0.  0.  1.  0.  0.
  1.  0.  0.  2.  0.  1.  1.  0.  0.  1.  0.  0.]
Difference in Hcount: [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Number of heavy atoms with different number of hydrogens:  3
Not a physical microstate pair.
Done!




,microstate ID 1,microstate ID 2,physical
0,SM18_micro001,SM18_micro002,False
1,SM18_micro001,SM18_micro003,False
2,SM18_micro001,SM18_micro004,False
3,SM18_micro001,SM18_micro005,False
4,SM18_micro001,SM18_micro007,False
5,SM18_micro001,SM18_micro009,False
6,SM18_micro001,SM18_micro010,False
7,SM18_micro001,SM18_micro011,False
8,SM18_micro001,SM18_micro013,True
9,SM18_micro001,SM18_micro016,True


In [6]:
df_physical_microstate_pairs = df_microstate_pairs.loc[df_microstate_pairs["physical"] == True]
df_physical_microstate_pairs = df_physical_microstate_pairs.reset_index(drop=True)

print(df_physical_microstate_pairs.shape[0])
df_physical_microstate_pairs

105


,microstate ID 1,microstate ID 2,physical
0,SM18_micro001,SM18_micro013,True
1,SM18_micro001,SM18_micro016,True
2,SM18_micro001,SM18_micro029,True
3,SM18_micro001,SM18_micro038,True
4,SM18_micro002,SM18_micro004,True
5,SM18_micro002,SM18_micro005,True
6,SM18_micro002,SM18_micro020,True
7,SM18_micro002,SM18_micro031,True
8,SM18_micro002,SM18_micro034,True
9,SM18_micro002,SM18_micro037,True


In [7]:
# write to csv file
microstate_pair_file_path = "./microstate_pairs/"+molecule_ID+"_microstate_pairs.csv"
df_physical_ms_pairs = df_physical_microstate_pairs.loc[:, ("microstate ID 1","microstate ID 2")]
df_physical_ms_pairs.to_csv(microstate_pair_file_path, index=False)